In [1]:
import numpy as np
import sys
sys.path.append('../')
from utils.sparse_vector import *
from utils.vector_polynomial_isomorphism import *
from utils.minor_flattening import *

We note that we are using the base field `QQ` for all computations currently

In [2]:
# Let us initialize constants and vector spaces.

d = 3
N = d+1
n = [N]
indeterminates = form_indeterminates(n)
R = PolynomialRing(QQ, indeterminates)
R.inject_variables()

glN = MatrixSpace(QQ, N)
dimV = binomial(N + 4 - 1, 4)
V = VectorSpace(QQ, dimV)

vp_iso = vector_polynomial_isomorphism(R, V, indeterminates, 4)

Defining x_0, x_1, x_2, x_3


In [3]:
f = x_0^2*x_3^2 - 6*x_0*x_1*x_2*x_3 + 4*x_1^3*x_3 + 4*x_0*x_2^3 - 3*x_1^2*x_2^2
f_vec = [vp_iso.polynomial_to_vector(fi) for fi in [f]] 

In [4]:
# check Tangential variety vanishes on f

R_parameterized = PolynomialRing(QQ, [var('t'), var('s')])
R_parameterized.inject_variables()
T_param = [1] + [R_parameterized(t^(i+1) + s*diff(t^(i+1), t)) for i in range(d)]
print(T_param)
print(f(T_param))


Defining t, s
[1, t + s, t^2 + 2*t*s, t^3 + 3*t^2*s]
0


In [5]:
W = V.subspace(f_vec)
print(W)

Vector space of degree 35 and dimension 1 over Rational Field
Basis matrix:
[ 0  0  0  0  0  0  0  0  0  1  0  0  0  0 -6  0  4  0  0  0  0  0  4 -3  0  0  0  0  0  0  0  0  0  0  0]


In [6]:
W_perp = W.complement()
print(W_perp)
p = W.dimension()
q = W_perp.dimension()

Vector space of degree 35 and dimension 34 over Rational Field
Basis matrix:
34 x 35 dense matrix over Rational Field


In [7]:
def E_action(a, b, grad_fl):
    sum = -1*indeterminates[b]*grad_fl[a]
    return R(sum)

We note `M_f` $:= \tilde{M}_f$

In [8]:
g_sparse = [sparsify(W_perp.gen(k)) for k in range(q)]

In [9]:
E_action_fl = [[0 for j in range(N)] for i in range(N)]

# Construct M_f
M_f = zero_matrix(QQ, p*q, N^2, sparse=True)
for l in range(p):
    grad_fl = vp_iso.vector_to_polynomial(W.gen(l)).gradient()
    for i in range(N):
        for j in range(N):
            E_action_fl[i][j] = sparsify(vp_iso.polynomial_to_vector(E_action(i,j,grad_fl)))
    for k in range(q):
        for i in range(N):
            for j in range(N):
                M_f[(q*l) + k,(N*i) + j] = sparse_inner_product(E_action_fl[i][j], g_sparse[k])
                

In [10]:
ns = M_f.right_kernel()
print('rank', M_f.rank())
print('dim(ker(M_f3)) =', ns.dimension())


rank 12
dim(ker(M_f3)) = 4


In [11]:
print('expected: 4')

expected: 4
